In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
tf.keras.backend.set_floatx('float64')

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
x_train.shape

(60000, 28, 28)

In [4]:
# Add a channels dimension
# 多增加第4個 dimension:"channel"
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [5]:
x_train.shape

(60000, 28, 28, 1)

In [6]:
#== tf.data API 來建立 input pipeline ==#
#  * tf.data

#1."from_tensor_slices" 建立一個 Dataset Object
#  在使用"from_tensor_slices"參數（x_train, y_train）是
#  包含很多筆類型一樣的data,像這個例子有 60000筆data。
#  不要跟"from_tensor"搞混了!

#2.將training data做shuffle打亂 和 將batch size設定為 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(10000).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(32)

In [7]:
#== 使用 Keras 自訂一個 Model ==#
#  * tf.Keras.Model
#  * tf.Keras.layers

#1.建立一個 Class並繼承 tf.Keras.Model
#2.__init__裡面我們可以自己設計每一層 Layer長什麼樣子，
#  是哪種 Layer(Convolution, dense...)

#3.call定義了這個 Model的 forward pass

class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()    
    # Define your layers here.
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    # Define your forward pass here,
    # using layers you previously defined (in __init__).
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [8]:
#== 選擇 loss, optimizer和 metrics ==#
#  * tf.keras.losses
#  * tf.keras.optimizers
#  * tf.keras.metrics


loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
#== train the model (定義)==#
#  * tf.function
#  * tf.GradientTape

@tf.function
def train_step(images, labels):
    
    #
    with tf.GradientTape() as tape:
        # forward pass to get predictions
        predictions = model(images)
        # compute loss with the ground truth and our predictions
        loss = loss_object(labels, predictions)
    # compute gradient 
    gradients = tape.gradient(loss, model.trainable_variables)
    # back propagation
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
#== test the model (定義)==#
#  * tf.function

@tf.function
def test_step(images, labels):
    
    # forward pass to get predictions
    predictions = model(images)
    # compute loss with the ground truth and our predictions
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [11]:
#== 真正開始 train model ==#

# 定義EPOCH數(整個dataset 拿來train過一次稱一個epoch)
EPOCH = 5
for epoch in range(EPOCH):
    for images, labels in train_dataset:
        train_step(images, labels)

    for test_images, test_labels in test_dataset:
        test_step(test_images, test_labels)    
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}\n'
    print(template.format(epoch+1,
                          train_loss.result(),
                          train_accuracy.result()*100,
                          test_loss.result(),
                          test_accuracy.result()*100))

    # Reset the metrics for the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


Epoch 1, Loss: 0.1361412711072713, Accuracy: 95.92833333333334, Test Loss: 0.06618688415461473, Test Accuracy: 97.98

Epoch 2, Loss: 0.04315773413044711, Accuracy: 98.61, Test Loss: 0.05517825220916372, Test Accuracy: 98.11999999999999

Epoch 3, Loss: 0.023053838163016673, Accuracy: 99.23666666666666, Test Loss: 0.06364900499247364, Test Accuracy: 98.03

Epoch 4, Loss: 0.014675820976573353, Accuracy: 99.49333333333334, Test Loss: 0.060037536181184646, Test Accuracy: 98.21

Epoch 5, Loss: 0.009271579573984248, Accuracy: 99.685, Test Loss: 0.05755405626680754, Test Accuracy: 98.46000000000001



In [12]:
#== Saved Model ==#
#  * tf.saved_model

tf.saved_model.save(model, "grandma")

W0920 00:04:12.716366 140127712978752 deprecation.py:506] From /home/leonardo/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1781: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
def evaluate(model):
    for images, labels in test_dataset:
        predictions = model(images)
        t_loss = loss_object(labels, predictions)

        test_loss(t_loss)
        test_accuracy(labels, predictions)
    
    print("test loss : {}".format(test_loss.result()))        
    print("test accuracy : {}".format(test_accuracy.result()))
    test_loss.reset_states()
    test_accuracy.reset_states()

In [18]:
#== Load Model ==#
# create a new model to see the performance without training

new_model = MyModel()
evaluate(new_model)

test loss : 2.2954691484713328
test accuracy : 0.1248


In [19]:
#== Load Model =#
#  Load 之前存起來的 Model再看看他的 performance
#  * tf.saved_model

new_model = tf.saved_model.load("grandma")
evaluate(new_model)


test loss : 0.05755476520098324
test accuracy : 0.9846


In [20]:
# 之前那個 Model
# performance 與上面的一樣！
evaluate(model)

test loss : 0.05755476520098324
test accuracy : 0.9846
